In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import yaml
import phacsl.utils.formats.csv_tools as csv_tools
import os.path
from phacsl.utils.collections.phacollections import SingletonMetaClass
import pyrheautils
import random
from collections import deque, defaultdict
from policybase import TransferDestinationPolicy as BaseTransferDestinationPolicy
from facilitybase import CareTier, tierToQueueMap
from tools_util import readModelInputs, getFacDict


In [ ]:
runDesc = '/home/welling/git/pyRHEA_github/src/sim/twoyear_allfac_OC.yaml'
inputDict = readModelInputs(runDesc)
facDict = getFacDict(inputDict)

# Drop annoying community entries
dropL = []
for abbrev, rec in facDict.items():
    if rec['category'] == 'COMMUNITY':
        dropL.append(abbrev)
for abbrev in dropL:
    del facDict[abbrev]

_constants = pyrheautils.importConstants(pyrheautils.pathTranslate('$(MODELDIR)/constants/indirecttransferdestination_constants.yaml'),
                                        '../schemata/indirecttransferdestination_constants_schema.yaml')
print _constants['transferFilePaths']

In [ ]:
with open('/home/welling/git/pyRHEA_github/src/sim/time_samples.yaml', 'r') as f:
    sampJSON = yaml.load(f)
sampD = {}
for ent in sampJSON:
    if ent['abbrev'] not in sampD:
        sampD[ent['abbrev']] = []
    if 'occupancy' in ent['samples'] and ent['time'] >= 800 and ent['time'] <= 830:
        sampD[ent['abbrev']].extend(ent['samples']['occupancy'])
print sampD['SJUD']
                                        

In [ ]:
print len(sampD['SJUD'])

In [ ]:
meanOccD = {}
for abbrev, valL in sampD.items():
    if not valL:
        continue
    meanV = sum(valL)/len(valL)
    meanOccD[abbrev] = meanV
#print meanOccD


In [ ]:
nBedsD = {}
for abbrev in sampD:
    nBedsD[abbrev] = facDict[abbrev]['nBeds']['value']
print nBedsD['SJUD']

In [ ]:
abbrevL = sampD.keys()
abbrevL.sort()
recs = []
keys = ['abbrev', 'nBeds', 'meanPop', 'occupancy']
for abbrev in abbrevL:
    occVal = meanOccD[abbrev] if abbrev in meanOccD else None
    recs.append({'abbrev': abbrev, 'nBeds': nBedsD[abbrev], 'meanPop': facDict[abbrev]['meanPop']['value'],
                 'occupancy': occVal})
with open('tmp.csv', 'w') as f:
    csv_tools.writeCSV(f, keys, recs)
                 

In [ ]:
sampD = defaultdict(list)
ratioTplL = []
for abbrev in abbrevL:
    cat = facDict[abbrev]['category']
    if abbrev in meanOccD and meanOccD[abbrev] is not None:
        ratio = float(meanOccD[abbrev])/float(nBedsD[abbrev])
        sampD[cat].append(ratio)
        ratioTplL.append((ratio, abbrev))
    #sampD[cat].append(float(meanOccD[abbrev]/float(facDict[abbrev]['meanPop']['value'])))
#print sampD
ratioTplL.sort()
print ratioTplL[-8:]

                      

In [ ]:
sampGpL = []
labelL = []
for k, v in sampD.items():
    sampGpL.append(v)
    labelL.append(k)

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
plt.rcParams["figure.figsize"] = [12, 9]
fig, axes = plt.subplots(1,1)
axes.hist(sampGpL, bins=20, stacked=True, label=labelL)
axes.set_ylabel('counts')
axes.set_xlabel('ratio of occupancy to num beds')
axes.set_title('Occupancy-to-beds ratio for each facility')
axes.legend()
plt.savefig('test.png')